In [5]:
import torch
import torch.nn as nn
import numpy as np
import tqdm

In [6]:
def parse_data_config(path: str):
    print(path)
    """데이터셋 설정 파일 분석"""
    options = {}
    with open(path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options

def load_classes(path: str):
    print(path)
    """클래스 이름 로드"""
    with open(path, "r") as f:
        names = f.readlines()
    for i, name in enumerate(names):
        names[i] = name.strip()
    return names

def init_weights_normal(m):
    """정규분포 형태로 가중치 초기화"""
    classname = m.__class__.__name__
    # https://discuss.pytorch.org/t/object-has-no-attribute-weight/31526
    # if classname.find("Conv") != -1:
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight.data, 0.1)

    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

def xywh2xyxy(x):
    y = x.new(x.shape)
    y[..., 0] = x[..., 0] - x[..., 2] / 2
    y[..., 1] = x[..., 1] - x[..., 3] / 2
    y[..., 2] = x[..., 0] + x[..., 2] / 2
    y[..., 3] = x[..., 1] + x[..., 3] / 2
    return y

def ap_per_class(tp, conf, pred_cls, target_cls):
    """
    Compute the average precision, given the Precision-Recall curve.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:    True positives (list).
        conf:  Objectness value from 0-1 (list).
        pred_cls: Predicted object classes (list).
        target_cls: True object classes (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Compute AP", leave=False):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  # Number of ground truth objects
        n_p = i.sum()  # Number of predicted objects

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    # Compute F1 score (harmonic mean of precision and recall)
    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")


def compute_ap(recall, precision):
    """
    Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def get_batch_statistics(outputs, targets, iou_threshold):
    """Compute true positives, predicted scores and predicted labels per batch."""
    batch_metrics = []
    for i, output in enumerate(outputs):

        if output is None:
            continue

        pred_boxes = output[:, :4]
        pred_scores = output[:, 4]
        pred_labels = output[:, -1]

        true_positives = np.zeros(pred_boxes.shape[0])

        annotations = targets[targets[:, 0] == i][:, 1:]
        target_labels = annotations[:, 0] if len(annotations) else []
        if len(annotations):
            detected_boxes = []
            target_boxes = annotations[:, 1:]

            for pred_i, (pred_box, pred_label) in enumerate(zip(pred_boxes, pred_labels)):

                # If targets are found break
                if len(detected_boxes) == len(annotations):
                    break

                # Ignore if label is not one of the target labels
                if pred_label not in target_labels:
                    continue

                iou, box_index = bbox_iou(pred_box.unsqueeze(0), target_boxes).max(0)
                if iou >= iou_threshold and box_index not in detected_boxes:
                    true_positives[pred_i] = 1
                    detected_boxes += [box_index]
        batch_metrics.append([true_positives, pred_scores, pred_labels])
    return batch_metrics


# 앵커 박스와 GT 박스 사이의 IOU 계산
# 모양만 본다(위치 필요 x)
def bbox_wh_iou(wh1, wh2):
    wh2 = wh2.t()
    w1, h1 = wh1[0], wh1[1]
    w2, h2 = wh2[0], wh2[1]
    inter_area = torch.min(w1, w2) * torch.min(h1, h2)
    union_area = (w1 * h1 + 1e-16) + w2 * h2 - inter_area
    return inter_area / union_area

def build_targets(pred_boxes, pred_cls, target, anchors, ignore_thres, device):
    # pred_boxes => (batch_size, anchor_num, gride, gride, 4)
    # pred_cls   => (batch_size, anchor_num, gride, gride, 80)
    nB = pred_boxes.size(0)   # batch 크기
    nA = pred_boxes.size(1)   # anchor box 개수 = 3
    nC = pred_cls.size(-1)    # class 개수 = 80
    nG = pred_boxes.size(2)   # gird 크기 = 13 or 26 or 52
    
    # output 초기화
    #(batch_size, anchor_num, gride, gride)
    obj_mask = torch.zeros(nB, nA, nG, nG, dtype=torch.bool, device=device)    # 물체인지 아닌지
    noobj_mask = torch.ones(nB, nA, nG, nG, dtype=torch.bool, device=device)
    class_mask = torch.zeros(nB, nA, nG, nG, dtype=torch.float, device=device)   # 어느 클래스인지
    iou_scores = torch.zeros(nB, nA, nG, nG, dtype=torch.float, device=device)
    tx = torch.zeros(nB, nA, nG, nG, dtype=torch.float, device=device)   # target x_ctr
    ty = torch.zeros(nB, nA, nG, nG, dtype=torch.float, device=device)   # target y_ctr
    tw = torch.zeros(nB, nA, nG, nG, dtype=torch.float, device=device)   # target w
    th = torch.zeros(nB, nA, nG, nG, dtype=torch.float, device=device)   # target h
    # (batch_size, anchor_num, gride, gride, class_num)
    tcls = torch.zeros(nB, nA, nG, nG, nC, dtype=torch.float, device=device)   # target class
    
    # 정규화된 4개의 변수를 실제 크기로 변환
    # target shape: [index, class, x_ctr, y_ctr, w, h]
    target_boxes = target[:, 2:6] * nG   # (batch_size, (x_ctr, y_ctr, w, h))
    gxy = target_boxes[:, :2]   # (batch, (x_ctr, y_ctr))
    gwh = target_boxes[:, 2:]   # (batch, (w, h))
    
    # print(gwh.shape)
    
    # 앵커 박스와 GT 박스 사이의 IOU 계산
    # 두 박스는 wh는 다르지만 중심점의 좌표는 동일하므로 wh만 필요
    # ious에는 총 3개의 anchor의 iou와 해당 anchor의 index가 들어있다. => (3, n)
    ious = torch.stack([bbox_wh_iou(anchor, gwh) for anchor in anchors]) 
    _, best_ious_idx = ious.max(0)   # iou가 가장 큰 anchor의 index
    
    b, target_labels = target[:, :2].long().t()   # batch-size(타겟박스의 개수), (index, class)
    gx, gy = gxy.t()
    gw, gh = gwh.t()
    gi, gj = gxy.long().t()   # 왼쪽 상단 모서리의 좌표
    
    # 마스크 설정 
    obj_mask[b, best_ious_idx, gj, gi] = 1     # 대상이 있을 것으로 추정되는 cell을 1로
    noobj_mask[b, best_ious_idx, gj, gi] = 0   # 대상이 있을 것으로 추정되는 cell을 0으로
    
    # IOU가 임계값 이상인 noobj 마스크를 0으로 설정(has-obj)
    for i, anchor_ious in enumerate(ious.t()):
        # ious.t() shape: [number of gt boxes, 3]
        noobj_mask[b[i], anchor_ious > ignore_thres, gj[i], gi[i]] = 0
        # b[i]: i번째 타겟박스
        
    # 타겟의 offset계산(cell에서의 위치 계산)
    tx[b, best_ious_idx, gj, gi] = gx - gx.floor()   # .floor(): 소수점 아래 무시
    ty[b, best_ious_idx, gj, gi] = gy - gy.floor()   # 결국 소수점 아래의 값만 남는다.
    tw[b, best_ious_idx, gj, gi] = torch.log(gw / anchors[best_ious_idx][:, 0] + 1e-16)
    th[b, best_ious_idx, gj, gi] = torch.log(gh / anchors[best_ious_idx][:, 1] + 1e-16)
    
    # target class label의 원핫인코딩 => 어떤 클래스인지
    # [b, best_ious_idx, gj, gi, target_labels]: b번째 타겟이 best_ious_idx번쨰 엥커를 사용해 객체의 유형(target_labels)을 예측
    tcls[b, best_ious_idx, gj, gi, target_labels] = 1
    
    # best Anchor에서의 label정확성 및 IOU 계산
    # [b, best_n, gj, gi] index를 가져 와서 이것이 참 값과 같은지 판단하고 올바른 index얻음
    class_mask[b, best_ious_idx, gj, gi] = (pred_cls[b, best_ious_idx, gj, gi].argmax(-1) == target_labels).float()
    # IOU점수 계산. 클수록 점수가 높다.
    iou_scores[b, best_ious_idx, gj, gi] = bbox_iou(pred_boxes[b, best_ious_idx, gj, gi], target_boxes, x1y1x2y2=False)
    
    # 타겟 신뢰도 계산
    tconf = obj_mask.float()
    
    return iou_scores, class_mask, obj_mask, noobj_mask, tx, ty, tw, th, tcls, tconf

def bbox_iou(box1, box2, x1y1x2y2=True):
    # xywh -> xyxy 
    if not x1y1x2y2:
        # 중심과 너비에서 정확한 좌표(꼭지점 좌표)로 변환
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
    else:
        # bounding box의 좌표 가져오기
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]
    # 교차 직사각형 좌표
    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    
    # 교집합 넒이
    # torch.calmp: min혹은 max의 범주에 해당하도록 값 변경
    # 1을 더하는 이유: 0부터 시작하는 픽셀 좌표를 보완하기 위함이라는 말도 있고, 0으로 나눠지는것을 막기 위함이라는 말도 있는데 둘 중 뭔지 잘 모르겠다.
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * torch.clamp(
        inter_rect_y2 - inter_rect_y1 + 1, min=0
    )
    # 합집합 넓이
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)
    union_area = b1_area + b2_area - inter_area
    
    # 0으로 나누는 것을 방지하는 작은 값
    epsilon = 1e-16
    
    # iou계산
    iou = inter_area / (union_area + epsilon)
    
    return iou
    
# test에서 사용
# NMS: 예측한 박스들을 score가 높은 순으로 정렬 후 가장 높은 박스와 IoU가 일정 이상인 박스는 동일한 물체를 detect했다고 판단해 지운다
# 10647개의 앵커에서 예측 결과를 계산한다.
def non_max_suppression(prediction, conf_thres, nms_thres):
    # From (center x(t_x), center y(t_y), width(t_w), height(t_h)) to (x1, y1, x2, y2)
    prediction[..., :4] = xywh2xyxy(prediction[..., :4])
    output = [None for _ in range(len(prediction))]
    
    for image_i, image_pred in enumerate(prediction):
        # 임계값보다 작은 confidence score 필터링
        # image_pred shape (10647,85)
        image_pred = image_pred[image_pred[:, 4] >= conf_thres]
        
        # 모든 prediction이 필터링 되면 다음 이미지로,,
        if not image_pred.size(0):
            continue
        
        # 신뢰도에 분류예측의 최대값을 곱한 점수를 score로 정의한다.
        # max(1): get max class (values, indices)
        # [0]: get max class values
        # category confidence = objectness x max_class_confidence 
        score = image_pred[:, 4] * image_pred[:, 5:].max(1)[0]
        
        # score가 높은 순으로 정렬한다.
        # argsort는 내림차순 정렬이므로 -score를 한다.
        image_pred = image_pred[(-score).argsort()]
        # 모든 객체 호출, 분류 예측 값이 가장 높은 상자에 해당하는 행 선택!(keepdim -> 원핫 인코딩)
        # 최대 클래스 신뢰도와 클래스 레이블을 얻는다.
        class_confs, class_preds = image_pred[:, 5:].max(1, keepdim=True)
        # 예측 상자와 score가 가장 높은 것 연결
        detections = torch.cat((image_pred[:, :5], class_confs.float(), class_preds.float()), 1)
        
        # NMS 수행
        keep_boxes = []
        while detections.size(0):
            # 첫 번쨰 상자와(score가장 높음) 모든 상자의 IOU를 계산하고 임계값보다 크면 1 아니면 0!
            # return (0, 0, 1, 0 ...)
            # unsqueeze(0): add dimension 
            large_overlap = bbox_iou(detections[0, :4].unsqueeze(0), detections[:, :4]) > nms_thres
            # 첫 번쨰 상자와(score가장 높음) 동일한 범주의 레이블을 가진 모든 예측상자 매치
            # return (0, 1, 0, 0 ...)
            label_match = detections[0, -1] == detections[:, -1]
            # iou가 임계값보다 크고, 동일한 범주의 label을 가지는 것들이 신뢰도를 가중치로 사용
            invalid = large_overlap & label_match
            weights = detections[invalid, 4:5]
            
            # 겹치는 상자의 좌표를 병합해 새로운 최적의 상자 좌표를 만든다.
            detections[0, :4] = (weights * detections[invalid, :4]).sum(0) / weights.sum()
            # 최적의 상자 유지
            keep_boxes += [detections[0]]
            # 이전 계산된 객체를 제외하고 다음 연산 수행
            detections = detections[~invalid]
        # 만일 keep_boxes가 None이 아닌 경우 스택의 모든 keep_box를 출력 목록에 저장
        if keep_boxes:
            output[image_i] = torch.stack(keep_boxes)

    return output
    # 출력의 형태: (batch_size, pred_boxes_num, 7)
    # 7: x, y, w, h, conf, class_conf, class_pred
    # pred_boxes_num: 각 사진에 pred_boxes_num 개의 상자가 있다는 것
